# Personal Project by Arvin Assanny

#### Import library 

In [63]:
import pandas as pd
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id = "hacktiv8-arvin" 
client = bigquery.Client(project=project_id)

Authenticated


## Assignment Problems
I am a data analyst working in the biggest e-commerce platform in planet Mars, which is known as The look. I was asked to make and give an evalutation report about our sales activity from the first until third quater of 2022.

### Problem Statement
The company wants to set a new sales target for the last quarter of 2022, which will be about 40% increase from total sales from last quarter. Unfortunately, they are not sure whether such goal could be achived or not. So they asked me to gather some findings about this problem, then give them some recomendations.

From this problem, we will use SMART methode.
* Specific: Do an overiew analysis on our sales data to get some insight on our data. From this data we can decide how much target we should 
set.

* Measurable: Get 2 table which present data about sales

* Attainable: Query data to return table which consist the total sales per quarter

* Relevant: To acquire data and set a new target for next quarter

* Time-bound: The end of the month


### Poin penjabaran:

1. Berapa jumlah transaksi yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022? Insight apa yang bisa kamu berikan?

In [22]:
# Query with SQL
df1 = client.query('''
--First, we want to extract the time format which will give us seperate year, month, and quarter
SELECT EXTRACT(YEAR FROM o.created_at) AS year, 
  EXTRACT(MONTH FROM o.created_at) AS month,
  EXTRACT(QUARTER FROM o.created_at) AS quarter, 
  COUNT(o.order_id) AS total_transc
-- Join this two table because both tables are connected by order_id column
FROM `bigquery-public-data.thelook_ecommerce.orders` AS o
JOIN `bigquery-public-data.thelook_ecommerce.order_items`AS oi
ON o.order_id = oi.order_id
-- Give some conditions to filter our result, according to the assignment
WHERE EXTRACT(YEAR FROM o.created_at) = 2022
  AND EXTRACT(QUARTER FROM o.created_at) IN (1,2,3)
  AND o.status = 'Complete'
GROUP BY year, month, quarter
ORDER BY year, month, quarter
''').to_dataframe()

# Calling dataframe 
df1

,year,month,quarter,total_transc
0,2022,1,1,1385
1,2022,2,1,1295
2,2022,3,1,1553
3,2022,4,2,1631
4,2022,5,2,1696
5,2022,6,2,1898
6,2022,7,3,2145
7,2022,8,3,2215
8,2022,9,3,2612


* Here we can see total of transaction in every month of 2022 from Q1 until Q3. We can see the total of transaction is almost increasing on every months, with an exception on February 2022, where the numbers are silghtly decreasing. This is a good thing that the customers are using our platform to buy their needs.

* We also know that on September 2022 has the highest total transaction per month when compared to another month from this Three Quarters of 2022, with total of 2612 transactions.

2. Berikan informasi total penjualan (dalam USD) tiap bulan selama Q1 sampai Q3 di tahun 2022! (Hanya yang transaksi berstatus Complete). Apa informasi yang bisa kamu sampaikan?



In [23]:
# Query with SQL
df2 = client.query('''
SELECT EXTRACT(YEAR FROM o.created_at) AS year,
  EXTRACT(MONTH FROM o.created_at) AS month,
  EXTRACT(QUARTER FROM o.created_at) AS quarter, 
  ROUND(SUM(oi.sale_price), 2) AS total_sale -- Using Round to make the result simpler
FROM `bigquery-public-data.thelook_ecommerce.orders` AS o
JOIN `bigquery-public-data.thelook_ecommerce.order_items`AS oi
ON o.order_id = oi.order_id
WHERE EXTRACT(YEAR FROM o.created_at) = 2022
  AND EXTRACT(QUARTER FROM o.created_at) IN (1,2,3)
  AND o.status = 'Complete'
GROUP BY year, month, quarter
ORDER BY year, month, quarter
''').to_dataframe()

# Calling dataframe 
df2

,year,month,quarter,total_sale
0,2022,1,1,84080.32
1,2022,2,1,80998.76
2,2022,3,1,92702.11
3,2022,4,2,96931.31
4,2022,5,2,101713.67
5,2022,6,2,111410.05
6,2022,7,3,127444.99
7,2022,8,3,126064.32
8,2022,9,3,156699.42


* Here we can see total of sales in every month of 2022 from Q1 until Q3. We can see the total of transaction is increasing almost on every months which means that the company revenue from is getting higher in the first 3 quarters of 2022, with an exception on February 2022 and August 2022 where the numbers are silghtly decreasing. 

* We also know that on September 2022 has the highest total sales or revenue per month when compared to another month from the first 3 quarters of 2022, with total revenue of $ 156,699.42.

3. Berapa user yang melakukan transaksi berstatus Complete di tiap bulan dari Q1 sampai Q3 2022? Apa kesimpulanmu?

In [24]:
# Query with SQL
df3 = client.query('''
SELECT EXTRACT(YEAR FROM o.created_at) AS year,
  EXTRACT(MONTH FROM o.created_at) AS month,
  EXTRACT(QUARTER FROM o.created_at) AS quarter, 
  COUNT(DISTINCT o.user_id) AS num_of_users -- Using distinct to show unique users
FROM `bigquery-public-data.thelook_ecommerce.orders` AS o
JOIN `bigquery-public-data.thelook_ecommerce.order_items`AS oi
ON o.order_id = oi.order_id
WHERE EXTRACT(YEAR FROM o.created_at) = 2022
  AND EXTRACT(QUARTER FROM o.created_at) IN (1,2,3)
  AND o.status = 'Complete'
GROUP BY year, month, quarter
ORDER BY year, month, quarter
''').to_dataframe()

# Calling dataframe 
df3

,year,month,quarter,num_of_users
0,2022,1,1,951
1,2022,2,1,892
2,2022,3,1,1077
3,2022,4,2,1111
4,2022,5,2,1150
5,2022,6,2,1337
6,2022,7,3,1443
7,2022,8,3,1522
8,2022,9,3,1780


* Here we can see total of unique users who made orders in 2022 from Q1 until Q3. We can see the total of unique users who made orders is almost increasing on every months with an exception on February 2022, where the numbers are silghtly decreasing. 

* We also know that on September 2022 has the highest total of unique users who made orders per month when compared to another month from the first 3 quarters of 2022, with total of 1780 unique users.

4. Kategori produk apa saja yang paling banyak dibeli (status transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

In [25]:
# Query with SQL
df4 = client.query('''

-- First, make CTE to filter data which will return the most bought product
WITH cte AS 
(
  SELECT EXTRACT(YEAR FROM delivered_at) AS year,
  EXTRACT(MONTH FROM delivered_at) AS month,
  EXTRACT(QUARTER FROM delivered_at) AS quarter,
  product_category, 
  COUNT(product_category) AS num_sales
-- We using inner join from these two table to get sales count from every product category 
FROM `bigquery-public-data.thelook_ecommerce.order_items`AS oi
JOIN `bigquery-public-data.thelook_ecommerce.inventory_items` AS ii
ON oi.inventory_item_id = ii.id
WHERE EXTRACT(YEAR FROM delivered_at) = 2022
  AND EXTRACT(QUARTER FROM delivered_at) IN (1,2,3)
  AND status = 'Complete'
GROUP BY year, month, quarter,product_category
ORDER BY year ASC, month ASC, quarter ASC, COUNT(product_category) DESC
),

-- Then we try to use window function to rank the result, we also make second CTE
filtered AS 
(
  SELECT *, 
 RANK() OVER(PARTITION BY month ORDER BY num_sales DESC) rn
FROM cte
)

-- Here we retrieve result by filtering rank = 1 from the second CTE
SELECT year, month, quarter, product_category, num_sales
FROM filtered
WHERE rn = 1
ORDER BY year ASC, month ASC, quarter ASC, num_sales DESC
''').to_dataframe()

# Calling dataframe 
df4

,year,month,quarter,product_category,num_sales
0,2022,1,1,Intimates,114
1,2022,2,1,Intimates,93
2,2022,3,1,Intimates,111
3,2022,4,2,Intimates,138
4,2022,5,2,Intimates,140
5,2022,6,2,Fashion Hoodies & Sweatshirts,143
6,2022,7,3,Intimates,151
7,2022,8,3,Intimates,156
8,2022,9,3,Jeans,181


* To get this result i try to make a temp table or a CTEs instead of subqueries because i find this method is easier to read, eventhough the process is more or less similar to subqueries.

* From this query, now we know that the most bought product category varies in every month.

* Intimates category has the most frequent appeared as the most bought product category in every month of first 3 quarters on 2022.

5. Kategori produk apa saja yang paling banyak mendapatkan pendapatan (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

In [26]:
# Query with SQL
df5 = client.query('''

-- First, make CTE to filter data which will return total sales of every product
WITH cte AS 
(SELECT EXTRACT(YEAR FROM delivered_at) AS year,
  EXTRACT(MONTH FROM delivered_at) AS month,
  EXTRACT(QUARTER FROM delivered_at) AS quarter,
  product_category, 
  ROUND(SUM(sale_price),2) AS total_sales
FROM `bigquery-public-data.thelook_ecommerce.order_items`AS oi
JOIN `bigquery-public-data.thelook_ecommerce.inventory_items` AS ii
ON oi.inventory_item_id = ii.id
WHERE EXTRACT(YEAR FROM delivered_at) = 2022
  AND EXTRACT(QUARTER FROM delivered_at) IN (1,2,3)
  AND status = 'Complete'
GROUP BY year, month, quarter,product_category
ORDER BY year ASC, month ASC, quarter ASC, SUM(sale_price) DESC
),

-- Then we try to use window function to rank the result, we also make second CTE
filtered AS 
(SELECT *, 
 RANK() OVER(PARTITION BY month ORDER BY total_sales DESC) rn
FROM cte)

-- Here we retrieve result by filtering rank = 1 from the second CTE
SELECT year, month, quarter, product_category, total_sales 
FROM filtered
WHERE rn = 1
ORDER BY year ASC, month ASC, quarter ASC, total_sales DESC
''').to_dataframe()

# Calling dataframe 
df5

,year,month,quarter,product_category,total_sales
0,2022,1,1,Outerwear & Coats,10332.81
1,2022,2,1,Outerwear & Coats,12776.95
2,2022,3,1,Outerwear & Coats,12223.63
3,2022,4,2,Jeans,12956.85
4,2022,5,2,Outerwear & Coats,13793.63
5,2022,6,2,Jeans,12512.62
6,2022,7,3,Outerwear & Coats,18648.70
7,2022,8,3,Jeans,14009.50
8,2022,9,3,Outerwear & Coats,18656.83


* We know that Outerwear & Coats product category has the most total sales product category from 3 quarter in 2022, with the most total sales on September 2022.

* From this query, we also know that the total sale of every product category varies in almost every month.

* The highest value of total sales from Outerwear & Coats product category is $ 18,656.83.

6. Dibandingkan dengan jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi? Apa analisis yang dapat kamu jelaskan? 

In [27]:
# Import library needed to calculate correaltion
import scipy.stats as stats

# Compare total transaction with total user
pearson_coef, p_value = stats.pearsonr(df1['total_transc'], df3['num_of_users'])
print("The Pearson Correlation Coefficient is", round(pearson_coef, 3), "with a P-value of P =", p_value)

The Pearson Correlation Coefficient is 0.998 with a P-value of P = 7.392499166365872e-10


* Using this function from `scipy.stats` libary we can get the correlation value from both columns. With correlation value of 0.998 between total transaction and total unique user, we can say that two variables has a positive realtion.

* Also, having the correlation value of 0.998 means that if one variable moves higher or lower, the other variable moves in the same direction.

In [28]:
# Compare total sales with total user
pearson_coef, p_value = stats.pearsonr(df2['total_sale'], df3['num_of_users'])
print("The Pearson Correlation Coefficient is", round(pearson_coef, 3), "with a P-value of P =", p_value)

The Pearson Correlation Coefficient is 0.992 with a P-value of P = 1.7203275330815116e-07


* With correlation value of 0.992 between total transaction and total unique user, we can say that two variables has a positive realtion. Also, having the correlation value of 0.992 means that if one variable moves higher or lower, the other variable moves in the same direction.

From knowing this data we can conclude that the comparison between `total transaction with total user` has higher correlation value, compared to `total sales with total user`, which means that total transaction has more corelation than total sales when compared to total of unique users who made their orders.

## Overall Analysis
After doing some analysis with the dataset, we can conclude that:

* September 2022 has the highest total transaction per month when compared to another month from this Three Quarters of 2022, with total of 2612 transactions.

* September 2022 has the highest total sales or revenue per month when compared to another month from the first 3 quarters of 2022, with total revenue of $ 156,699.42.

* September 2022 has the highest total of unique users who made orders per month when compared to another month from the first 3 quarters of 2022, with total of 1780 unique users.

* Intimates category has the most frequent appeared as the most bought product category in every month of first 3 quarters on 2022.

* Outerwear & Coats product category has the most total sales product category from 3 quarter in 2022

* Total transation has more corelation than total sales when compared to total of unique users who made their orders.


## Poin Pertanyaan

1. Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!

Yes, it can be measured using the SMART method we use earlier in problem statement. Because by using this me method we already set what are our success factors in fullfiling our objective. Our objective in this task was to know whether if the company could set the 40% increase in total sales target in the last quarter of 2022, by doing some overview analysis now we know that the target be achived by current sales activiy. Therefore, it might be applied.

Here is the calculation:

In [61]:
# Query with SQL
a = client.query('''
SELECT EXTRACT(YEAR FROM o.created_at) AS year,
  EXTRACT(QUARTER FROM o.created_at) AS quarter, 
  ROUND(SUM(oi.sale_price), 2) AS total_sale -- Using Round to make the result simpler
FROM `bigquery-public-data.thelook_ecommerce.orders` AS o
JOIN `bigquery-public-data.thelook_ecommerce.order_items`AS oi
ON o.order_id = oi.order_id
WHERE EXTRACT(YEAR FROM o.created_at) = 2022
  AND EXTRACT(QUARTER FROM o.created_at) IN (1,2,3)
  AND o.status = 'Complete'
GROUP BY year, quarter
ORDER BY year, quarter
''').to_dataframe()

# Calling dataframe 
a

,year,quarter,total_sale
0,2022,1,257781.19
1,2022,2,310055.03
2,2022,3,410208.73


Here we can see our total sale per quarter.

In [62]:
a['diff'] = a['total_sale'].diff()
a['Percentage_Change'] = (a['total_sale'].pct_change() * 100).round(2)
a

,year,quarter,total_sale,diff,Percentage_Change
0,2022,1,257781.19,NaN,NaN
1,2022,2,310055.03,52273.84,20.28
2,2022,3,410208.73,100153.70,32.30


We can see that there is a trend that our sales are increasing in every quarter. So the target of 40% increase in the next quarter is possible. 

2. Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai $250000, apakah masuk akal?

First we will show the data frame

In [64]:
df2

,year,month,quarter,total_sale
0,2022,1,1,84080.32
1,2022,2,1,80998.76
2,2022,3,1,92702.11
3,2022,4,2,96931.31
4,2022,5,2,101713.67
5,2022,6,2,111410.05
6,2022,7,3,127444.99
7,2022,8,3,126064.32
8,2022,9,3,156699.42


From this table we can see that on September 2022 the total sales is USD 156,699.42. So, if the target for the early months Q4 is USD250,000 it might not be possible to achieve. 

Here is the calculation:

In [65]:
df2['diff'] = df2['total_sale'].diff()
df2

,year,month,quarter,total_sale,diff
0,2022,1,1,84080.32,NaN
1,2022,2,1,80998.76,-3081.56
2,2022,3,1,92702.11,11703.35
3,2022,4,2,96931.31,4229.20
4,2022,5,2,101713.67,4782.36
5,2022,6,2,111410.05,9696.38
6,2022,7,3,127444.99,16034.94
7,2022,8,3,126064.32,-1380.67
8,2022,9,3,156699.42,30635.10


Here the differences between total sales in every month, next we want to see the average of increase from total sales.

In [68]:
df2['diff'].mean().round(2)

9077.39

Here the average increase is USD 9077.39. So it might not be possible to achive the target of such magnitude.

3. CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!

To know whether if our company eligible for new round of funding, we have to give the informnation about trends of total transaction, total unique users, and GMV or total revenue from the last 3 quarter.

Here we will calculate using pandas.

In [74]:
# Avarage increase of total transaction per month from the last 3 quarters
df1['diff'] = df1['total_transc'].diff()
df1['diff'].mean().round(0)

153.0

In [75]:
# Avarage increase of total sales per month from the last 3 quarters
df3['diff'] =df3['num_of_users'].diff()
df3['diff'].mean().round(0)

104.0

In [72]:
# Avarage increase of total sales per month from the last 3 quarters
# Since we already calculate the difference, so we can go ahead to calculate the average
df2['diff'].mean().round(2)

9077.39

From this calculations, we know the potential of from our company where it is possible that we grow more bigger. This is shown from trends of total transaction to increase 153 transaction per month, total users who purchase to increase 104 users per month , and GMV to increase about USD 9077,39 per month.

Therefore, our company is eligible for fundings from the new investor.